# Final Model


In [1]:
import matplotlib.pyplot as plt
from pathlib import Path
from matplotlib.collections import LineCollection
import matplotlib as mpl
import numpy as np
from scipy import optimize as opt
from scipy import integrate as intg
from scipy.optimize import least_squares
from scipy.stats import beta
from scipy.stats import cumfreq, beta

%matplotlib inline
plt.style.use('../notebook.mplstyle')

## Exogenous Parameters


In [2]:
# Setting exogenous parameters
def reset_exog_params():
    global Bm, Bw, bm_vals, bw_vals, δ, Um, Uw, Fm, Fw, λm, λw  
    Bm = 10
    Bw = 10
    bm_vals = range(1,Bm+1) 
    bw_vals = range(1,Bw+1)
    
    δ = 0.97
    Um = lambda θ : θ 
    Uw = lambda θ : θ
    Fm = beta(3,3)
    Fw = beta(3,3)
    λm = 60
    λw = 10

## Two-Sided Search Equilibrium Conditions 


In [3]:
# Optimality conditions
def SSE(x): 
    # Endogenous Variables 
    μ = x[:Bm] 
    ω = x[Bm:Bm+Bw]   

    Pm = x[Bm+Bw:2*Bm+Bw]
    Pw = x[2*Bm+Bw:2*Bm+2*Bw]  

    Nm = x[2*Bm+2*Bw] 
    Nw = x[2*Bm+2*Bw+1]  
    
    τm = min(Nw/Nm, 1)
    τw = τm * (Nm/Nw) 
    αm = (τm*δ)/(1-δ*(1-τm))
    αw = (τw*δ)/(1-δ*(1-τw)) 

    # Initialising system of equilibrium equations
    E = np.empty(2*Bm + 2*Bw + 2) 

    # Initial condition for men 
    E[0] = (Um(μ[0]) 
            - αm * Um(μ[0]) * Fw.cdf(μ[0]) 
            - αm * intg.quad(lambda t: Um(t) * Fw.pdf(t), μ[0], 1)[0]) 
    
    # Intertemporal optimality conditions for men
    for b in range(1, Bm):
        E[b] = (Um(μ[b]) 
                - αm * Um(μ[b]) * Fw.cdf(μ[b]) 
                - αm * Um(μ[b-1])*(1-Fw.cdf(μ[b-1])) 
                - αm * intg.quad(lambda t : Um(t) * Fw.pdf(t), μ[b], μ[b-1])[0])
    
    # Initial condition for women     
    E[Bm] = (Uw(ω[0]) 
            - αw * Uw(ω[0]) * Fm.cdf(ω[0]) 
            - αw * intg.quad(lambda t: Uw(t) * Fm.pdf(t), ω[0], 1)[0])

    # Intertemporal optimality conditions for women
    for b in range(1, Bw):
        E[Bm+b] = (Uw(ω[b]) 
                - αw * Uw(ω[b]) * Fm.cdf(ω[b]) 
                - αw * Uw(ω[b-1])*(1-Fm.cdf(ω[b-1])) 
                - αw * intg.quad(lambda t : Uw(t) * Fm.pdf(t), ω[b], ω[b-1])[0]) 
                
    # Platform flow men
    E[Bm+Bw] = λm - (1-δ)*Nm - Nm*Pm[0]*δ*τm*(1-Fw.cdf(μ[0]))

    for b in range(0, Bm-1):
       E[Bm+Bw+b+1] = (Nm*Pm[b+1]*δ*τm*(1-Fw.cdf(μ[b+1]))
                    - Nm*Pm[b]*(1-δ) - Nm*Pm[b]*δ*τm*(1-Fw.cdf(μ[b])))

    E[2*Bm+Bw] = λm - Nm*Pm[Bm-1]*(1-δ) - Nm*Pm[Bm-1]*δ*τm*(1-Fw.cdf(μ[Bm-1]))

    # Platform flow women
    E[2*Bm+Bw+1] = λw - (1-δ)*Nw - Nw*Pw[0]*δ*τw*(1-Fm.cdf(ω[0]))  

    for b in range(0, Bw-1):
       E[2*Bm+Bw+2+b] = (Nw*Pw[b+1]*δ*τw*(1-Fm.cdf(ω[b+1]))
                      - Nw*Pw[b]*(1-δ) - Nw*Pw[b]*δ*τw*(1-Fm.cdf(ω[b])))

    E[2*Bm+2*Bw+1] = λw - Nw*Pw[Bw-1]*(1-δ) - Nw*Pw[Bw-1]*δ*τw*(1-Fm.cdf(ω[Bw-1])) 

    return E 

## Solving For Steady State Equilibria 

In [4]:
reset_exog_params()
μ_test = np.random.rand(Bm) 
ω_test = np.random.rand(Bw) 

Pm_test = np.random.rand(Bm)
Pw_test = np.random.rand(Bw) 

N_test = np.array([λm*(Bm+1), λw*(Bw+1)])

print('μ0: ', μ_test)
print('ω0: ', ω_test)
print('')

x_start = np.concatenate((μ_test, ω_test, Pm_test, Pw_test, N_test), axis=None) 
solution = opt.root(SSE, x_start) 

μ0:  [0.03295406 0.77120375 0.44856717 0.55329347 0.78084813 0.40156185
 0.14450108 0.12133605 0.20323418 0.98812986]
ω0:  [0.24749592 0.25403478 0.3699439  0.05870985 0.92768268 0.28595342
 0.19893785 0.91058033 0.29186971 0.95674151]



In [12]:
μ_star = solution.x[:Bm]
ω_star = solution.x[Bm:Bm+Bw]
loss = round(np.sum(SSE(solution.x)**2), 3)
print('μ* ', μ_star) 
print('ω* ', ω_star) 
print('Loss: ',loss) 
print('')

Pm = solution.x[Bm+Bw:2*Bm+Bw]
Pw = solution.x[2*Bm+Bw:2*Bm+2*Bw]  

Nm = solution.x[2*Bm+2*Bw] 
Nw = solution.x[2*Bm+2*Bw+1]

# Normalized masses 
nm = [(Pm[b]*Nm)/(Nm+Nw) for b in range(Bm)]
nw = [(Pw[b]*Nw)/(Nm+Nw) for b in range(Bw)]

print('Masses: ', round((Nm)/(Nm+Nw),3), round((Nw)/(Nm+Nw),3))
print('Unity Sum check:', round(sum(Pm),3), round(sum(Pw),3))
print('')

τm = min(Nw/Nm, 1)
τw = τm * (Nm/Nw) 
αm = (τm*δ)/(1-δ*(1-τm))
αw = (τw*δ)/(1-δ*(1-τw)) 
print('Tightness: ', round(τm,3), round(τw,3))
print('Alphas: ', round(αm,3), round(αw,3)) 
print('')

ρm = sum([(1 - Fm.cdf(ω_star[b]))*Pw[b] for b in range(Bw)])
ρw = sum([(1 - Fw.cdf(μ_star[b]))*Pm[b] for b in range(Bm)])
print('Average Pr(right-swipe): ', round(ρm, 3), round(ρw, 3))

μ*  [0.38847937 0.26440119 0.18787829 0.13545192 0.09820325 0.07135752
 0.05189726 0.03775768 0.02747439 0.01999287]
ω*  [0.66366994 0.59515949 0.54890106 0.51301625 0.4833236  0.45781311
 0.43534878 0.41521996 0.3969497  0.38020096]
Loss:  0.0

Masses:  0.924 0.076
Unity Sum check: 1.0 1.0

Tightness:  0.083 1.0
Alphas:  0.728 0.97

Average Pr(right-swipe):  0.491 0.988


In [11]:
# Plotting Marginal PDFs
fig, axs = plt.subplots(2,3,figsize=(6.267704441677044, 3.2824362509300642), constrained_layout=True)
x = np.linspace(0, 1, 5000)

axs[0,0].set_xlim(0,1)
axs[0,0].set_ylim(0,1.1*max(Fm.pdf(x)))
axs[0,0].plot(x, Fm.pdf(x), color='tab:blue') 

axs[1,0].set_xlim(0,1)
axs[1,0].set_ylim(0,1.1*max(Fw.pdf(x)))
axs[1,0].plot(x, Fw.pdf(x), color='tab:pink')

# Plotting Marginal PMFs 
axs[0,1].set_xlim(0,Bm+1)
axs[0,1].bar(bm_vals, nm, color='tab:blue') 

axs[1,1].set_xlim(0,Bw+1)
axs[1,1].bar(bw_vals, nw, color='tab:pink') 

axs[0,1].set(ylabel=r'$N_m(b)$') 
axs[1,1].set(xlabel=r'Budget ($b$)', ylabel=r'$N_w(b)$') 

#axs[0,0].set_title('Men',loc='center')
axs[0,0].set(ylabel=r'$f_m(\theta)$')

#axs[0,1].set_title('Women',loc='center')
axs[1,0].set(xlabel=r'Attractiveness ($\theta$)', ylabel=r'$f_w(\theta)$')


for i in range(2):
    B = Bm if i==0 else Bw
    b_vals = bm_vals if i==0 else bw_vals
    θ_vals = μ_star if i==0 else ω_star
    
    axs[i,2].set_xlim(1,B)
    axs[i,2].set_ylim(0,1)
    axs[i,2].plot(b_vals, θ_vals, color='k')
    axs[i,2].fill_between(b_vals, 0, θ_vals, color='red', alpha=0.05)
    axs[i,2].fill_between(b_vals, θ_vals, 1, color='green', alpha=0.1)

axs[0,2].set(ylabel=r'Attractiveness ($\theta$)')
axs[1,2].set(xlabel=r'Budget ($b$)', ylabel=r'Attractiveness ($\theta$)') 
#plt.savefig('../../figures/mkt-cs.png', bbox_inches='tight')
plt.show()